In [ ]:
URL = 'http://localhost:3000'

In [ ]:
import requests
from requests_toolbelt.multipart.encoder import MultipartEncoder

In [ ]:
def FileExists(filename):
    try:
        with open(filename, 'r') as f:
            return True
    except FileNotFoundError:
        return False

In [ ]:
import enum

class RequestType(enum.Enum):
    GET = 0
    POST = 1
    PUT = 2
    DELETE = 3
    
class RequestBuilder:
    def __init__(self, url):
        self.url = url
        self.headers = {'Accept': 'application/json'}
        self.query = {}
        self.fields = {}
        self.token = None
        self.isMultipart = False
    
    def SetToken(self, token):
        self.token = token
        return self
        
    def AddHeader(self, key, value):
        self.headers[key] = str(value)
        return self
    
    def AddBody(self, key, value):
        self.fields[key] = str(value)
        return self
        
    def AddQuery(self, key, value):
        self.query[key] = str(value)
        return self
    
    def GetFile(self, path):
        if not FileExists(path):
            raise FileNotFoundError(path)
        
        filename = path.split('/')[-1]
        
        filetype = filename.split('.')[-1].lower()
        mimetype = None
        if filetype == 'jpg' or filetype == 'jpeg':
            mimetype = 'image/jpeg'
            
        if filetype == 'png':
            mimetype = 'image/png'
        
        if not mimetype:
            raise Exception('Unsupported file type')
        
        return (filename, open(path, 'rb'), mimetype)
    
    def AddFile(self, fieldname, path):
        # self.files.append((fieldname, self.GetFile(path)))
        self.fields[fieldname] = self.GetFile(path)
        self.isMultipart = True
        return self
        
    def Build(self, request_type):
        new_headers = self.headers.copy()
        if (self.token):
            new_headers['Authorization'] = 'Bearer ' + self.token
 
        if self.isMultipart and request_type in [RequestType.POST, RequestType.PUT]:
            encoder = MultipartEncoder(fields=self.fields)
            new_headers['Content-Type'] = encoder.content_type
            if request_type == RequestType.POST:
                return requests.post(self.url, headers=new_headers, params=self.query, data=encoder)
            elif request_type == RequestType.PUT:
                return requests.put(self.url, headers=new_headers, params=self.query, data=encoder)
            
        if request_type == RequestType.GET:
            return requests.get(self.url, headers=new_headers, params=self.query)
        elif request_type == RequestType.POST:
            return requests.post(self.url, headers=new_headers, params=self.query, data=self.fields)
        elif request_type == RequestType.PUT:
            return requests.put(self.url, headers=new_headers, params=self.query, data=self.fields)
        elif request_type == RequestType.DELETE:
            return requests.delete(self.url, headers=new_headers, params=self.query)
        raise Exception('Unsupported request type')
    
    def GET(self):
        return self.Build(RequestType.GET)
    
    def POST(self):
        return self.Build(RequestType.POST)
    
    def PUT(self):
        return self.Build(RequestType.PUT)
    
    def DELETE(self):
        return self.Build(RequestType.DELETE)
        

In [ ]:
def login(username, password):

    res = (RequestBuilder(URL+'/auth/login')
           .AddBody('username', username)
           .AddBody('password', password)
           .POST())
    token = None
    if (res.status_code in [200, 201]):
        print(f'{username} login success')
        token = res.json()['access_token']
    else:
        print(f'{username} login failed')
        if (res.json()['message'] == 'User not found'):
            # raise Exception('User not found')
            return None
        else:
            print(res.json())
            exit(1)
    return token

In [ ]:
class TokenManager:
    __STORE_TOKEN = None
    __STORE_USERNAME = 'abcdavid'
    __STORE_PASSWORD = '123456'
    __STORE_ID = None
    
    @staticmethod
    def __init_store():
        TokenManager.__STORE_TOKEN = login(TokenManager.__STORE_USERNAME, TokenManager.__STORE_PASSWORD)
        if not TokenManager.__STORE_TOKEN:
            CreateUserRequest = (RequestBuilder(URL+'/users/createuser')
                                .AddBody('username', TokenManager.__STORE_USERNAME)
                                .AddBody('password', TokenManager.__STORE_PASSWORD)
                                .AddBody('email', TokenManager.__STORE_USERNAME + '@gmail.com')
                                .POST())
            if (CreateUserRequest.status_code not in [200, 201]):
                print('Create account failed')
            TokenManager.__STORE_TOKEN = login(TokenManager.__STORE_USERNAME, TokenManager.__STORE_PASSWORD)
                
        RegisterStoreRequest = (RequestBuilder(URL+'/product/store')
                                .AddBody('name', 'Abcdavid Store')
                                .AddBody('description', 'Store of Abcdavid')
                                .AddBody('address', '0')
                                .AddFile('logo', './CategoryImages/Boy-Accessories.png')
                                .AddHeader('Content-Type', 'multipart/form-data; charset=utf-8')
                                .SetToken(TokenManager.__STORE_TOKEN)
                                .POST())
        if (RegisterStoreRequest.status_code not in [200, 201]):
            print('Register store failed')
            exit(1)
        TokenManager.__STORE_ID = RegisterStoreRequest.json()['id']
    
    @staticmethod
    def StoreId():
        if not TokenManager.__STORE_ID:
            TokenManager.__init_store()
        return TokenManager.__STORE_ID
    
    @staticmethod
    def StoreToken():
        if not TokenManager.__STORE_TOKEN:
            TokenManager.__init_store()
        return TokenManager.__STORE_TOKEN

In [ ]:
def calc_indent(numb: int) -> str:
    if numb < 2:
        return ""
    binary = bin(numb)[3:]
    ret = map(lambda x: "│  " if x == "1" else "   ", binary[:-1])
    return "".join(ret) + ( "└──" if binary[-1] == "0" else "├──")

class Category:
    def __init__(self, name, description, image):
        self.id = None
        self.name = name
        self.description = description if description else None
        self.children = []
        self.image = image
        self.same_name = True
        self.same_description = True
        
    def from_json(json):
        cat = Category(json['name'], json['description'], json['image'])
        cat.id = json['id']
        for child in json['children']:
            cat.children.append(Category.from_json(child))
        return cat
        
    def add_child(self, child):
        self.children.append(child)
    
    def add_child(self, name, description, image):
        self.children.append(Category(name, description, image))
        
    def add_children(self, children):
        self.children.extend(map(lambda x: 
            Category(x[0],x[1],x[2])
            if isinstance(x, tuple) 
            else Category(x, None, None)
            , children))
        
    def image_exists(self):
        if self.image:
            return FileExists('./CategoryImages/' + self.image)
        return False
        
    def __str__(self, indent=1):
        ret = calc_indent(indent) + self.__repr__() + "\n"
        for child in self.children[:-1]:
            ret += child.__str__(indent*2+1)
        if len(self.children) > 0:
            ret += self.children[-1].__str__(indent*2)
        return ret
    
    def __eq__(self, __value: object) -> bool:
        if isinstance(__value, Category):
            return self.name == __value.name
        else:
            return False
        
    def match(self, target: object):
        if not self == target:
            return
        self.id = target.id
        if not self.description and target.description:
            self.description = target.description
        if self.name != target.name:
            self.same_name = False
        if self.description != target.description:
            self.same_description = False
        for child in self.children:
            for target_child in target.children:
                if child == target_child:
                    child.match(target_child)
        
    def short_desc(self):
        if self.description:
            return self.description[:20] + "..." if len(self.description) > 20 else self.description
        else:
            return ""
    
    def __repr__(self):
        return f'[{self.id if self.id else "?"}] {"!" if not self.same_name else ""}{self.name} - {"!" if not self.same_description else ""}"{self.short_desc()}" {"!" if not self.image_exists() else ""}[{self.image if self.image else "None"}]'

    def get_child(self, name):
        for child in self.children:
            if child.name == name:
                return child
        return None

    def get_name_by_id(self, id):
        if self.id == id:
            return self.name
        for child in self.children:
            name = child.get_name_by_id(id)
            if name:
                return self.name + '/' + name
        return None

    def __getitem__(self, key):
        if hasattr(self, 'children') and isinstance(self.children, (list, dict)):
            return self.get_child(key)
        else:
            raise AttributeError("The 'children' attribute does not exist or is not subscriptable.")

In [ ]:
class ServerCategory:
    
    __categories = {}
    __fetch = False
    
    @staticmethod
    def get_categories(cls):
        res = (RequestBuilder(URL+'/product/category/all')
               .GET())
        if res.status_code != 200:
            print(res).json()
            exit()
        for cat in res.json():
            cls.__categories[cat['name']] = Category.from_json(cat)
        return cls.__categories
    
    @staticmethod
    def __get_categories(cls):
        if not cls.__fetch:
            cls.__categories = cls.get_categories(cls)
            # cls.__fetch = True
        return cls.__categories
    
    @staticmethod
    def all():
        cats = []
        for cat in ServerCategory.__get_categories(ServerCategory).values():
            cats.append(cat)
        return cats
    
    def __class_getitem__(cls, key):
        return cls.__get_categories(cls).get(key)
    

In [ ]:
print(ServerCategory.all())
print(ServerCategory['Men'])

In [ ]:
print(ServerCategory['Women'])

Add Product

In [ ]:
from enum import Enum

class Size(Enum):
    XS = 'XS'
    S = 'S'
    M = 'M'
    L = 'L'
    XL = 'XL'
    XXL = 'XXL'

class Color(Enum):
    WHITES = 'Whites'
    BLACKS = 'Blacks'
    GREYS = 'Greys'
    BEIGES = 'Beiges'
    BROWNS = 'Browns'
    REDS = 'Reds'
    GREENS = 'Greens'
    BLUES = 'Blues'
    PURPLES = 'Purples'
    YELLOWS = 'Yellows'
    PINKS = 'Pinks'
    ORANGES = 'Oranges'

In [ ]:
def findCategoryIdByName(name):
    names = name.split('/')
    try:
        if len(names) == 1:
            return ServerCategory[names[0]].id
        elif len(names) == 2:
            return ServerCategory[names[0]][names[1]].id
        elif len(names) == 3:
            return ServerCategory[names[0]][names[1]][names[2]].id
        else:
            raise Exception("Invalid category name")
    except AttributeError as e:
        if "'NoneType' object has no attribute 'id'" in e.args:
            return None
        else:
            raise e

def getCategoryNameById(id):
    for cat in ServerCategory.all():
        name = cat.get_name_by_id(id)
        if name:
            return name
    return None

class ProductVariant:
    def __init__(self, product, color, size, price, quantity, image):
        self.id = None
        self.product = product
        self.color = color
        self.size = size
        self.price = price
        self.quantity = quantity
        self.image = image
    
    def __repr__(self):
        return f'{self.color} - {self.size} - {self.price} - {self.quantity} - {self.image}'
    
    def image_exists(self):
        if self.image:
            return FileExists('./ProductImages/' + self.image)
        return False
    
    def update(self):
        request = RequestBuilder(URL+'/product/variant').SetToken(TokenManager.StoreToken())
        if self.product:
            request.AddQuery('product', self.product)
        if self.color:
            request.AddQuery('color', self.color)
        if self.size:
            request.AddQuery('size', self.size)
        if self.price:
            request.AddBody('price', self.price)
        if self.quantity:
            request.AddBody('quantity', self.quantity)
        if self.image:
            request.AddFile('image', './ProductImages/' + self.image)
        res = request.POST()
        if res.status_code not in [200, 201]:
            print(f'Update product variant failed [{self.product}]')
            print(res.json())
            # raise Exception(f'[{res.status_code}] Failed to update product variant {self.product}')
        self.id = res.json()['id']
        print (f'[{self.id}] Updated {self.product} - {self.color} - {self.size} - {self.price} - {self.quantity}')        

class Product:
    def __init__(self, name, description, category, image):
        self.name = name
        self.description = description
        self.same_description = False
        self.category = category
        self.image = image
        self.variant = []
        
        self.id = None
        cat = findCategoryIdByName(self.category)
        if cat:
            self.category_id = cat
        else:
            raise Exception(f'Category {self.category} not found')
        self.fetch()
        
    @staticmethod
    def all():
        return (RequestBuilder(URL+'/product/store')
                .AddQuery('id', TokenManager.StoreId())
                .GET()).json()['products']
        
    def fetch(self):
        products = Product.all()
        product = next((product for product in products if product['name'] == self.name and product['category']['id'] == self.category_id), None)
        if product:
            self.id = product['id']
            self.same_description = self.description == product['description']
    
    def add_variant(self, color, size, price, quantity, image):
        self.variant.append(ProductVariant(self.id, color, size, price, quantity, image))
        
    def add_variant_without_image(self, color, size, price, quantity):
        self.variant.append(ProductVariant(self.id, color, size, price, quantity, self.image))
    
    def __repr__(self):
        return f'[{self.id if self.id else "?"}] {"!" if not self.same_description else ""}{self.name} - "{self.description}" {"!" if not self.image_exists() else ""}[{self.image if self.image else "None"}]'
    
    def __str__(self) -> str:
        ret = self.__repr__() + "\n"
        for variant in self.variant[:-1]:
            ret += "├──" + variant.__repr__() + "\n"
        if len(self.variant) > 0:
            ret += "└──" + self.variant[-1].__repr__()
        return ret
    
    def image_exists(self):
        if self.image:
            return FileExists('./ProductImages/' + self.image)
        return False
    
    def update(self):
        if not self.id:
            self.__add()
        else:
            request = RequestBuilder(URL+'/product').SetToken(TokenManager.StoreToken()).AddQuery('id', self.id)
            if self.name:
                request.AddBody('name', self.name)
            if self.description:
                request.AddBody('description', self.description)
            if self.category_id:
                request.AddBody('category', self.category_id)
            if self.image:
                request.AddFile('image', './ProductImages/' + self.image)
            res = request.PUT()
            if res.status_code not in [200, 201]:
                print(f'Update product failed [{self.name}]')
                print(res.json())
                # raise Exception(f'[{res.status_code}] Failed to update product {self.name}')
            print (f'[{res.status_code}] Updated {self.name} - "{self.description}"')

        for variant in self.variant:
            print(f'Updating variant {variant}')
            variant.update()
            
    def __add(self):
        if self.id:
            print(f'Product {self.name} already exists')
            return
        if not self.name:
            raise Exception('Product name is required') 
        if not self.category_id:
            raise Exception('Product category is required')
        if not self.image or not self.image_exists():
            raise Exception('Product image is required')
        request = (RequestBuilder(URL+'/product/')
                   .SetToken(TokenManager.StoreToken())
                   .AddBody('name', self.name)
                   .AddBody('description', self.description)
                   .AddBody('category', self.category_id)
                   .AddFile('image', './ProductImages/' + self.image)
                   .POST())
        if request.status_code not in [200, 201]:
            print(f'Add product failed [{self.name}]')
            print(request.json())
            # raise Exception(f'[{request.status_code}] Failed to add product {self.name}')
        
    
    
    @staticmethod
    def from_json(json):
        return Product(json['name'], json['description'], json['category'], json['image'])

In [ ]:
getCategoryNameById(28)

In [ ]:
findCategoryIdByName('Men/Shoes and accessories/Shoes')

In [ ]:
p = Product(
    name='Product 1',
    description='Product 1 description',
    category='Men',
    image='1.png'
)
p.add_variant_without_image('Reds', 'M', '50', '100')
p.add_variant_without_image('Reds', 'L', '200', '200')
p.add_variant_without_image('Reds', 'XL', '20', '200')
p.update()

In [ ]:
print(p)

In [ ]:
import pandas as pd
import os

data_folder = "./Data/Category/"  # replace with your actual folder path
files = os.listdir(data_folder)

dfs = []
for file in files:
    if file.endswith('.csv'):
        df = pd.read_csv(os.path.join(data_folder, file))
        dfs.append(df)

df = pd.concat(dfs, ignore_index=True).sort_values('id', ascending=True)

In [ ]:
findCategoryIdByName('Men/Clothing/Suits/Trousers')

In [51]:
i = 0
for row in df.values.tolist():
    i += 1
    if i > 5:
        break
    try:
        print('CategoryID:',findCategoryIdByName(row[2]))
    except Exception as e:
        print('CategoryID invalid:', row[2], '--'*50)
    print(row)

CategoryID: 3
[1, 'Handmade recycled wool double-breasted coat', 'Men/Clothing/Coats', 4899000]
CategoryID: 3
[2, 'Long recycled wool coat', 'Men/Clothing/Coats', 5499000]
CategoryID: 3
[3, 'Long recycled wool coat', 'Men/Clothing/Coats', 5499000]
CategoryID: 3
[4, 'Long recycled wool coat', 'Men/Clothing/Coats', 5499000]
CategoryID: 3
[5, 'Reversible recycled wool trench coat', 'Men/Clothing/Coats', 4999000]


In [ ]:
Products = []
for row in df.values.tolist():
    Products.append(Product(
        name=row[1],
        description=row[1],
        category=row[2],
        image=row[3]
    ))

In [52]:
product_data_folder = "./Data/Product/"  # replace with your actual folder path

pdf = pd.read_csv(os.path.join(data_folder, '1' + '.csv'))


FileNotFoundError: [Errno 2] No such file or directory: './Data/Category/1.csv'

In [ ]:
df[df['id'] % 2 == 1]
a = df[df['id'] % 2 == 1].values.tolist()
a

In [ ]:
Products = []
